## Human Face Expression Recognition 

#### In this one we would have the code for the model to train and we would save the model and use it and detect human expressions

In [1]:

import keras

Using TensorFlow backend.


In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os

In [4]:
base_dir='/home/jenil/Downloads'

In [5]:
train_dir=base_dir+'/'+'train'
validation_dir=base_dir+'/'+'validation'

In [7]:
print('len_of_types',len(os.listdir(validation_dir)))

len_of_types 6


#### Here we would also use the image augmentation technique to increase our accuracy

In [9]:
print('len 0f types in train',len(os.listdir(train_dir)))

len 0f types in train 6


In [10]:
train_datagen=ImageDataGenerator(rescale=1/255,
                                rotation_range=30,
                                shear_range=0.3,
                                zoom_range=0.3,
                                width_shift_range=0.4,
                                height_shift_range=0.4,
                                horizontal_flip=True,
                                fill_mode='nearest')

In [11]:
validation_datagen=ImageDataGenerator(rescale=1/255)

#### Here in the validation generator we are not using the image augmentation as it would increase our training time

In [12]:
train_generator=train_datagen.flow_from_directory(train_dir,
                                                 color_mode='grayscale',
                                                 batch_size=32,
                                                 target_size=(48,48),
                                                 class_mode='categorical',
                                                 )

Found 24256 images belonging to 5 classes.


In [13]:
validation_generator=validation_datagen.flow_from_directory(validation_dir,
                                                           color_mode='grayscale',
                                                           batch_size=32,
                                                           target_size=(48,48),
                                                           class_mode='categorical')

Found 3006 images belonging to 5 classes.


In [14]:
model=Sequential()

### Here we would be having our model divided into 6 parts
namely
- Conv Layer -1
- conv layer -2
- Conv layer -3
- Flatten layer
- Dense layer 1
- Final Output layer

In [15]:
#conv-layer-1
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(48,48,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(48,48,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [16]:
#conv-layer-2
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


In [17]:
#conv-layer-3
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#### Here we are using dropouts in every layer as it would help us in not to overfit the model

In [18]:
# Flatten Layer
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


In [19]:
#Dense Layer 1
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

In [21]:
#output layer
model.add(Dense(5,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)       

In [24]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


### Here we are going to define our call-backs which are important for training large examples

In [25]:
checkpoint = ModelCheckpoint('Emotion_little_vgg.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

In [26]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.001),
             metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,
                 steps_per_epoch=24256 //32,
                 epochs=13,
                 callbacks=callbacks,
                 validation_data=validation_generator,
                 validation_steps=3006//32)

Epoch 1/13
758/758 [==============================] - 1062s 1s/step - loss: 1.7701 - accuracy: 0.2468 - val_loss: 1.6320 - val_accuracy: 0.2802

Epoch 00001: val_loss improved from inf to 1.63204, saving model to Emotion_little_vgg.h5
Epoch 2/13
758/758 [==============================] - 810s 1s/step - loss: 1.5828 - accuracy: 0.2810 - val_loss: 1.5517 - val_accuracy: 0.3164

Epoch 00002: val_loss improved from 1.63204 to 1.55170, saving model to Emotion_little_vgg.h5
Epoch 3/13
758/758 [==============================] - 880s 1s/step - loss: 1.5486 - accuracy: 0.3025 - val_loss: 1.5734 - val_accuracy: 0.3278

Epoch 00003: val_loss did not improve from 1.55170
Epoch 4/13
757/758 [============================>.] - ETA: 1s - loss: 1.5214 - accuracy: 0.3198